# Khai báo thư viện

In [276]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import keras as kr
from keras.models import Sequential
from keras.layers import Dense
from sklearn import metrics


#Bước 1: Đọc file csv

In [277]:
data = pd.read_csv('/content/Churn_Modelling.csv')
data = data.drop(['CustomerId', 'Surname', 'RowNumber'], axis = 1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


#Bước 2: Chuyển đổi dữ liệu dạng không thứ tự sang dạng onehot encoding

In [278]:
X = data.iloc[:,0:10]
y = data.iloc[:,10]

In [279]:
X = pd.get_dummies(X)
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,0,1,0,0,1


#Chia tập dữ liệu train và test theo tỉ lệ 8 : 2

In [280]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [281]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(8000, 13)
(8000,)
(2000, 13)
(2000,)


#Bước 4: Chuẩn hóa bằng Standard Scaler

In [282]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)


#Bước 5: Build mạng Neural Network với kiến trúc: Input - 6 neuron/ReLU - 6 neuron/ReLU - 1 neuron/sigmoid

In [283]:
model = Sequential()
n_features = X_train.shape[1]
# model.add(Dense(6, activation='relu'))
model.add(Dense(6, input_dim=n_features , activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#Bước 6: Huấn luyện với thuật toán Adam và hàm độ lỗi Binary Cross Entropy

In [284]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [285]:
model.fit(X_train, y_train, epochs=50, batch_size=10)

Epoch 1/50
800/800 [==============================] - 2s 1ms/step - loss: 0.5297 - accuracy: 0.7750
Epoch 2/50
800/800 [==============================] - 1s 1ms/step - loss: 0.4356 - accuracy: 0.8105
Epoch 3/50
800/800 [==============================] - 1s 1ms/step - loss: 0.4188 - accuracy: 0.8230
Epoch 4/50
800/800 [==============================] - 1s 1ms/step - loss: 0.4054 - accuracy: 0.8292
Epoch 5/50
800/800 [==============================] - 1s 1ms/step - loss: 0.3840 - accuracy: 0.8434
Epoch 6/50
800/800 [==============================] - 1s 1ms/step - loss: 0.3650 - accuracy: 0.8512
Epoch 7/50
800/800 [==============================] - 1s 1ms/step - loss: 0.3568 - accuracy: 0.8550
Epoch 8/50
800/800 [==============================] - 1s 1ms/step - loss: 0.3530 - accuracy: 0.8558
Epoch 9/50
800/800 [==============================] - 1s 1ms/step - loss: 0.3501 - accuracy: 0.8558
Epoch 10/50
800/800 [==============================] - 1s 1ms/step - loss: 0.3489 - accuracy: 0.8589

#Bước 7: Test và đánh giá độ chính xác của mô hình

In [286]:
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

250/250 [==============================] - 0s 976us/step - loss: 0.3326 - accuracy: 0.8640
Accuracy: 86.40


#Thử nghiệm xem với khách hàng có thông tin sau thì có nên exit hay ko?

In [287]:
new_prediction = model.predict(scaler.transform(np.array([[600, 40 , 3, 60000, 2, 1, 1, 50000, 1, 0, 0, 0, 1]])))

new_prediction = (new_prediction > 0.5 )
print(new_prediction)

[[False]]


#Kfold


In [288]:
kfold = KFold(n_splits = 5, random_state= None)

In [289]:
def get_model():

  model = Sequential()
  n_features = X_train.shape[1]
  # model.add(Dense(6, activation='relu'))
  model.add(Dense(6, input_dim=n_features , activation='relu'))
  model.add(Dense(6, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

In [290]:
  X = np.concatenate((X_train, X_test), axis=0)
  y = np.concatenate((y_train, y_test), axis=0)

In [291]:
# Định nghĩa K-Fold CV
kfold = KFold(n_splits=5, shuffle=True)

loss_list =[]
accuracy_list = []
# K-fold Cross Validation model evaluation
fold_idx = 1

for train_ids, val_ids in kfold.split(X, y):

  model = get_model()

  print("Bắt đầu train Fold ", fold_idx)

  # Train model
  model.fit(X[train_ids], y[train_ids],
              batch_size=10,
              epochs= 50,
              verbose=1)

  # Test và in kết quả
  scores = model.evaluate(X[val_ids], y[val_ids], verbose=0)
  print("Đã train xong Fold ", fold_idx)

  # Thêm thông tin accuracy và loss vào list
  accuracy_list.append(scores[1] * 100)
  loss_list.append(scores[0])

  # Sang Fold tiếp theo
  fold_idx = fold_idx + 1

Bắt đầu train Fold  1
Epoch 1/50
800/800 [==============================] - 1s 1ms/step - loss: 93.9876 - accuracy: 0.7671
Epoch 2/50
800/800 [==============================] - 1s 1ms/step - loss: 13.9480 - accuracy: 0.7705
Epoch 3/50
800/800 [==============================] - 1s 1ms/step - loss: 8.0578 - accuracy: 0.7725
Epoch 4/50
800/800 [==============================] - 1s 1ms/step - loss: 9.3018 - accuracy: 0.7769
Epoch 5/50
800/800 [==============================] - 1s 1ms/step - loss: 6.6222 - accuracy: 0.7824
Epoch 6/50
800/800 [==============================] - 1s 1ms/step - loss: 8.1365 - accuracy: 0.7886
Epoch 7/50
800/800 [==============================] - 1s 1ms/step - loss: 5.9342 - accuracy: 0.7918
Epoch 8/50
800/800 [==============================] - 1s 1ms/step - loss: 6.8364 - accuracy: 0.7937
Epoch 9/50
800/800 [==============================] - 1s 1ms/step - loss: 5.1155 - accuracy: 0.8012
Epoch 10/50
800/800 [==============================] - 1s 1ms/step - loss: 4

In [292]:
print('* Chi tiết các fold')
for i in range(0, len(accuracy_list)):
  print(f'> Fold {i+1} - Loss: {loss_list[i]} - Accuracy: {accuracy_list[i]}%')

print('* Đánh giá tổng thể các folds:')
print(f'> Accuracy: {np.mean(accuracy_list)} (Độ lệch +- {np.std(accuracy_list)})')
print(f'> Loss: {np.mean(loss_list)}')

* Chi tiết các fold
> Fold 1 - Loss: 0.39024990797042847 - Accuracy: 83.85000228881836%
> Fold 2 - Loss: 0.41033610701560974 - Accuracy: 82.8499972820282%
> Fold 3 - Loss: 3.1317598819732666 - Accuracy: 79.14999723434448%
> Fold 4 - Loss: 0.3745162785053253 - Accuracy: 85.39999723434448%
> Fold 5 - Loss: 0.3879263699054718 - Accuracy: 84.60000157356262%
* Đánh giá tổng thể các folds:
> Accuracy: 83.16999912261963 (Độ lệch +- 2.179129149626397)
> Loss: 0.9389577090740204
